# tensorflow pistachio


In [1]:
import tensorflow as tf
print(tf.__version__)

2024-04-10 17:42:04.811603: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.16.1


## arff to csv





In [2]:
import pandas as pd 
from scipy.io import arff
import os 
label_mapping = {'Kirmizi_Pistachio': 0, 'Siit_Pistachio': 1}

def load_arff_file(input_arff: str) -> pd.DataFrame:
    """convert arff file to parquet"""
    if not os.path.exists(input_arff):
        raise ValueError(f"input file '{input_arff}' does not exist")
    print(f'loading arff file {input_arff}')
    data, meta = arff.loadarff(input_arff)
    print(f"arff metadata: {meta}")
    df = pd.DataFrame(data)
    df['Class'] = df['Class'].astype(str).map(label_mapping)
    
    return df
##################
arff_filename = './data/Pistachio_16_Features_Dataset.arff'
csv_filename = './data/pistachio_16.csv'
if not os.path.exists(csv_filename):
    df = load_arff_file(arff_filename)
    df.head()
    df.to_csv(csv_filename, index=False, header=True)
    print(f'wrote file to {csv_filename}')
else:
    print(f'{csv_filename} exists')


./data/pistachio_16.csv exists


## dataset


In [3]:
import numpy as np
def split_csv_data(infilename: str, train_filename: str, test_filename:str, test_fraction: float):
    df = pd.read_csv(infilename, header=0)
    columns = df.columns
    df['split_var'] = np.random.uniform(size=len(df))
    train_df = df.loc[df.split_var <= test_fraction][columns]
    test_df = df.loc[df.split_var > test_fraction][columns]
    train_df.to_csv(train_filename, index=False, header=True)
    test_df.to_csv(test_filename, index=False, header=True)
    print(f'wrote {len(train_df)} records to {train_filename}')
    print(f'wrote {len(test_df)} records to {test_filename}')
    
train_filename = './data/pistachio_train.csv'
test_filename = './data/pistachio_test.csv'

if not (os.path.exists(train_filename) and os.path.exists(test_filename)):
    split_csv_data(csv_filename, train_filename, test_filename, 0.2)
else:
    print(f'{train_filename} and {test_filename} exist')






./data/pistachio_train.csv and ./data/pistachio_test.csv exist


In [4]:
def map_func(features, labels):
    return tf.stack([features[k] for k in features]), labels

# use dataset.map to concatenate feature dictionary into tensor
pistachio_train_batches = tf.data.experimental.make_csv_dataset(
    train_filename, batch_size=4,
    num_epochs=1,
    label_name="Class").map(map_func)
pistachio_test_batches = tf.data.experimental.make_csv_dataset(
    test_filename, batch_size=4,
    num_epochs=1,
    label_name="Class").map(map_func)

In [5]:
# batch = 0
# for feature_batch, label_batch in pistachio_train_batches.take(2):
#     # print(f'{batch}, {label_batch.shape}')
#     # cat_batch = tf.stack([feature_batch['AREA'],feature_batch['PERIMETER']],axis=1)
#     # cat_batch = tf.stack([feature_batch[k] for k in feature_batch],axis=1)

#     # batch += 1
    
#     print("'label': {}".format(label_batch))
#     print(cat_batch)
#     print("features:")
#     for key, value in feature_batch.items():
#         print("  {!r:20s}: {}".format(key, value))

'label': [0 1 0 0]
tf.Tensor(
[[7.268200e+04 1.228455e+03 4.833133e+02 1.946325e+02 9.153000e-01
  3.042065e+02 9.681000e-01 7.507500e+04 7.105000e-01 2.483200e+00
  6.052000e-01 6.294000e-01 6.600000e-03 2.700000e-03 3.962000e-01
  9.838000e-01]
 [9.512000e+04 1.239937e+03 4.801718e+02 2.537507e+02 8.490000e-01
  3.480094e+02 9.779000e-01 9.726700e+04 7.501000e-01 1.892300e+00
  7.775000e-01 7.248000e-01 5.000000e-03 2.700000e-03 5.253000e-01
  9.940000e-01]
 [7.998200e+04 1.218160e+03 4.685190e+02 2.203987e+02 8.824000e-01
  3.191179e+02 9.689000e-01 8.255200e+04 6.927000e-01 2.125800e+00
  6.773000e-01 6.811000e-01 5.900000e-03 2.800000e-03 4.639000e-01
  9.862000e-01]
 [7.246000e+04 1.187195e+03 4.437118e+02 2.151141e+02 8.746000e-01
  3.037416e+02 9.519000e-01 7.612400e+04 7.555000e-01 2.062700e+00
  6.460000e-01 6.845000e-01 6.100000e-03 3.000000e-03 4.686000e-01
  9.666000e-01]], shape=(4, 16), dtype=float32)
features:
  'AREA'              : [72682. 95120. 79982. 72460.]
  'PER

2024-04-10 17:42:20.374779: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Model

In [6]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization
from tensorflow.keras import Model

class PistachioModel(Model):
  def __init__(self):
    super().__init__()
    self.bn = BatchNormalization()
    self.d1 = Dense(10, activation='relu')
    self.d2 = Dense(10)
    self.lout = Dense(1, activation='sigmoid')

  def call(self, x):
    x = self.bn(x)
    x = self.d1(x)
    x = self.d2(x)
    return self.lout(x)

# Create an instance of the model
model = PistachioModel()

## Keras model.fit api

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy', 'auc'])







In [8]:
model.fit(pistachio_train_batches, epochs=10)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:1210: UserWarning: Layer 'pistachio_model' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encoutered: ''-1''
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:359: UserWarning: `build()` was called on layer 'pistachio_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, 

KeyError: "Exception encountered when calling PistachioModel.call().\n\n\x1b[1m-1\x1b[0m\n\nArguments received by PistachioModel.call():\n  • x=OrderedDict([('AREA', 'tf.Tensor(shape=(None,), dtype=float32)'), ('PERIMETER', 'tf.Tensor(shape=(None,), dtype=float32)'), ('MAJOR_AXIS', 'tf.Tensor(shape=(None,), dtype=float32)'), ('MINOR_AXIS', 'tf.Tensor(shape=(None,), dtype=float32)'), ('ECCENTRICITY', 'tf.Tensor(shape=(None,), dtype=float32)'), ('EQDIASQ', 'tf.Tensor(shape=(None,), dtype=float32)'), ('SOLIDITY', 'tf.Tensor(shape=(None,), dtype=float32)'), ('CONVEX_AREA', 'tf.Tensor(shape=(None,), dtype=float32)'), ('EXTENT', 'tf.Tensor(shape=(None,), dtype=float32)'), ('ASPECT_RATIO', 'tf.Tensor(shape=(None,), dtype=float32)'), ('ROUNDNESS', 'tf.Tensor(shape=(None,), dtype=float32)'), ('COMPACTNESS', 'tf.Tensor(shape=(None,), dtype=float32)'), ('SHAPEFACTOR_1', 'tf.Tensor(shape=(None,), dtype=float32)'), ('SHAPEFACTOR_2', 'tf.Tensor(shape=(None,), dtype=float32)'), ('SHAPEFACTOR_3', 'tf.

In [ ]:
## custom training loop stuff

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result():0.2f}, '
    f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
    f'Test Loss: {test_loss.result():0.2f}, '
    f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'
  )